In [ ]:
from pyspark.sql import SparkSession

Loading MovieLens Data

File uploaded to /FileStore/tables/links-3.csv
File uploaded to /FileStore/tables/movies-3.csv
File uploaded to /FileStore/tables/tags-3.csv
File uploaded to /FileStore/tables/ratings-3.csv


In [ ]:
movies_df = spark.read.csv("/FileStore/tables/movies-3.csv", header=True, inferSchema=True)
ratings_df = spark.read.csv("/FileStore/tables/ratings-3.csv", header=True, inferSchema=True)
tags_df = spark.read.csv("/FileStore/tables/tags-3.csv", header=True, inferSchema=True)
links_df = spark.read.csv("/FileStore/tables/links-3.csv", header=True, inferSchema=True)


 Show first 5 rows of each dataset

In [ ]:

print("Movies DataFrame:")
movies_df.show(5)

print("Ratings DataFrame:")
ratings_df.show(5)

print("Tags DataFrame:")
tags_df.show(5)

print("Links DataFrame:")
links_df.show(5)


Movies DataFrame:
+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows
Ratings DataFrame:
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows
Tags DataFrame:
+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+-----

In [ ]:
Handle Missing Values

In [ ]:
from pyspark.sql.functions import col

def check_missing_values(df, name):
    print(f"Missing values in {name}:")
    df.select([col(c).isNull().alias(c) for c in df.columns]).show()

check_missing_values(movies_df, "movies_df")
check_missing_values(ratings_df, "ratings_df")
check_missing_values(tags_df, "tags_df")
check_missing_values(links_df, "links_df")


Missing values in movies_df:
+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
|  false|false| false|
+-------+-----+------+
only showing top 20 rows
Missing values in ratings_df:
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
| false|  false| false|    false|
| false|  false| false|    false|
| false|  false| false|    false|
| false|  false| false|    false|
| false|  false| false|    false|
| false|  false| false|    false|
| false|  false| false|    false|
| false|  false| false| 

Remove Rows with Missing Data

In [ ]:
movies_df = movies_df.dropna(subset=["title", "genres"])
ratings_df = ratings_df.dropna(subset=["userId", "movieId", "rating"])
tags_df = tags_df.dropna(subset=["userId", "movieId", "tag"])
links_df = links_df.dropna(subset=["movieId", "imdbId", "tmdbId"])


Remove Duplicate Data



In [ ]:
movies_df = movies_df.dropDuplicates()
ratings_df = ratings_df.dropDuplicates()
tags_df = tags_df.dropDuplicates()
links_df = links_df.dropDuplicates()


Fix Incorrect Data Types

In [ ]:
movies_df.printSchema()
ratings_df.printSchema()
tags_df.printSchema()
links_df.printSchema()


root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- imdbId: integer (nullable = true)
 |-- tmdbId: integer (nullable = true)



Convert Columns to Correct Data Types

In [ ]:
from pyspark.sql.types import IntegerType, DoubleType

ratings_df = ratings_df.withColumn("userId", col("userId").cast(IntegerType())) \
                       .withColumn("movieId", col("movieId").cast(IntegerType())) \
                       .withColumn("rating", col("rating").cast(DoubleType()))

links_df = links_df.withColumn("movieId", col("movieId").cast(IntegerType())) \
                   .withColumn("imdbId", col("imdbId").cast(IntegerType())) \
                   .withColumn("tmdbId", col("tmdbId").cast(IntegerType()))

Detect & Remove Outliers


In [ ]:
from pyspark.sql.functions import mean, stddev

stats = ratings_df.select(mean(col("rating")).alias("mean"), stddev(col("rating")).alias("stddev")).collect()[0]
mean_rating = stats["mean"]
stddev_rating = stats["stddev"]

ratings_df = ratings_df.filter((col("rating") >= mean_rating - 3 * stddev_rating) & 
                               (col("rating") <= mean_rating + 3 * stddev_rating))


Correct Spelling 

In [ ]:
from pyspark.sql.functions import lower, trim

movies_df = movies_df.withColumn("title", trim(lower(col("title"))))
tags_df = tags_df.withColumn("tag", trim(lower(col("tag"))))


Validate Cleaned Data

In [ ]:
movies_df.show(5)
ratings_df.describe().show()
tags_df.show(5)
links_df.show(5)


+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      4|waiting to exhale...|Comedy|Drama|Romance|
|      5|father of the bri...|              Comedy|
|      3|grumpier old men ...|      Comedy|Romance|
|      2|      jumanji (1995)|Adventure|Childre...|
|      6|         heat (1995)|Action|Crime|Thri...|
+-------+--------------------+--------------------+
only showing top 5 rows
+-------+------------------+-----------------+------------------+--------------------+
|summary|            userId|          movieId|            rating|           timestamp|
+-------+------------------+-----------------+------------------+--------------------+
|  count|            100836|           100836|            100836|              100836|
|   mean|326.12756356856676| 19435.2957177992| 3.501556983616962|1.2059460873684695E9|
| stddev|182.61849146349994|35530.98719870024|1.0425292390606402|2.162610

Converting Timestamp to Readable Format

In [ ]:
from pyspark.sql.functions import from_unixtime

ratings_df = ratings_df.withColumn("timestamp", from_unixtime(ratings_df.timestamp))
ratings_df.show(5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|     50|   5.0|2000-07-30 18:48:51|
|     1|      3|   4.0|2000-07-30 18:20:47|
|     1|      1|   4.0|2000-07-30 18:45:03|
|     1|     70|   3.0|2000-07-30 18:40:00|
|     1|      6|   4.0|2000-07-30 18:37:04|
+------+-------+------+-------------------+
only showing top 5 rows


Save Cleaned Data 

In [ ]:
movies_df.write.format("delta").mode("overwrite").saveAsTable("cleaned_movies")
ratings_df.write.format("delta").mode("overwrite").saveAsTable("cleaned_ratings")
tags_df.write.format("delta").mode("overwrite").saveAsTable("cleaned_tags")
links_df.write.format("delta").mode("overwrite").saveAsTable("cleaned_links")
